In [2]:
import pybullet as p
import pybullet_data
import argparse
import numpy as np
import os
import os.path as osp
import joblib
from scipy.spatial.transform import Rotation as R
from tqdm import tqdm
import sys
from contextlib import contextmanager
from multiprocessing import Pool
import time
import plotly
import plotly.graph_objs as go

pybullet build time: Nov 28 2023 23:45:17


In [3]:
result = joblib.load('/home/jianrenw/mocap/data/out/amass.pt')

In [4]:
keys = list(result.keys())
current_key = keys[10]
# current_key = 'DanceDB_20151003_AndriaMichaelidou_Andria_Afraid_v1_C3D_poses'
skeleton = result[current_key]['skeleton']
mocap_rate = result[current_key]['mocap_framerate']

In [10]:
num_frames = skeleton.shape[0]

pelvis = skeleton[:,0,:]

l_upperarm = skeleton[:,16,:]
l_forearm = skeleton[:,18,:]
l_hand = skeleton[:,20,:]

r_upperarm = skeleton[:,17,:]
r_forearm = skeleton[:,19,:]
r_hand = skeleton[:,21,:]

l_thigh = skeleton[:,1,:]
l_calf = skeleton[:,4,:]
l_foot = skeleton[:,7,:]
l_toe = skeleton[:,10,:]

r_thigh = skeleton[:,2,:]
r_calf = skeleton[:,5,:]
r_foot = skeleton[:,8,:]
r_toe = skeleton[:,11,:]

l_elbow_i = skeleton[:,24,:]
l_elbow_o = skeleton[:,25,:]
r_elbow_i = skeleton[:,26,:]
r_elbow_o = skeleton[:,27,:]

# upper body
head = (l_upperarm + r_upperarm) / 2
upper_z = (head - pelvis) / np.linalg.norm(head - pelvis, axis=1, keepdims=True)
upper_x = np.cross(pelvis - l_upperarm, pelvis - r_upperarm)
upper_x = upper_x / np.linalg.norm(upper_x, axis=1, keepdims=True)
upper_y = np.cross(upper_z, upper_x)
upper_rot = np.stack([upper_x, upper_y, upper_z], axis=2)

# lower body
crotch = (l_thigh + r_thigh) / 2
lower_z = - (crotch - pelvis) / np.linalg.norm(crotch - pelvis, axis=1, keepdims=True)
lower_x = np.cross(pelvis - r_thigh, pelvis - l_thigh)
lower_x = lower_x / np.linalg.norm(lower_x, axis=1, keepdims=True)
lower_y = np.cross(lower_z, lower_x)
lower_rot = np.stack([lower_x, lower_y, lower_z], axis=2)

# left arm
l_elbow_rot_z = l_forearm - l_hand
l_elbow_rot_z = l_elbow_rot_z / np.linalg.norm(
    l_elbow_rot_z, axis=1, keepdims=True
)
l_elbow_rot_y_1 = l_elbow_i - l_elbow_o
l_elbow_rot_y_1 = l_elbow_rot_y_1 / np.linalg.norm(
    l_elbow_rot_y_1, axis=1, keepdims=True
)
l_elbow_rot_y_2 = l_upperarm - l_forearm
l_elbow_rot_y_2 = l_elbow_rot_y_2 / np.linalg.norm(
    l_elbow_rot_y_2, axis=1, keepdims=True
)
l_elbow_rot_y_2 = np.cross(l_elbow_rot_y_2, -l_elbow_rot_z)
l_elbow_rot_y_2 = l_elbow_rot_y_2 / np.linalg.norm(
    l_elbow_rot_y_2, axis=1, keepdims=True
)
correction = np.einsum('ij,ij->i', l_elbow_rot_y_1, l_elbow_rot_y_2) < 0
l_elbow_rot_y_2[correction] = -l_elbow_rot_y_2[correction]

l_elbow_rot_x = np.cross(l_elbow_rot_y_2, l_elbow_rot_z)
l_elbow_rot_x = l_elbow_rot_x / np.linalg.norm(
    l_elbow_rot_x, axis=1, keepdims=True
)

l_elbow_rot = np.stack([l_elbow_rot_x, l_elbow_rot_y_2, l_elbow_rot_z], axis=2)

l_upperarm_dir = l_forearm - l_upperarm
l_upperarm_dir = l_upperarm_dir / np.linalg.norm(
    l_upperarm_dir, axis=1, keepdims=True
)

# right arm
r_elbow_rot_z = r_forearm - r_hand
r_elbow_rot_z = r_elbow_rot_z / np.linalg.norm(
    r_elbow_rot_z, axis=1, keepdims=True
)
r_elbow_rot_y_1 = r_elbow_i - r_elbow_o
r_elbow_rot_y_1 = r_elbow_rot_y_1 / np.linalg.norm(
    r_elbow_rot_y_1, axis=1, keepdims=True
)
r_elbow_rot_y_2 = r_upperarm - r_forearm
r_elbow_rot_y_2 = r_elbow_rot_y_2 / np.linalg.norm(
    r_elbow_rot_y_2, axis=1, keepdims=True
)   
r_elbow_rot_y_2 = np.cross(r_elbow_rot_y_2, -r_elbow_rot_z)
r_elbow_rot_y_2 = r_elbow_rot_y_2 / np.linalg.norm(
    r_elbow_rot_y_2, axis=1, keepdims=True
)
correction = np.einsum('ij,ij->i', r_elbow_rot_y_1, r_elbow_rot_y_2) < 0
r_elbow_rot_y_2[correction] = -r_elbow_rot_y_2[correction]

r_elbow_rot_x = np.cross(r_elbow_rot_y_2, r_elbow_rot_z)
r_elbow_rot_x = r_elbow_rot_x / np.linalg.norm(
    r_elbow_rot_x, axis=1, keepdims=True
)
r_elbow_rot = np.stack([r_elbow_rot_x, r_elbow_rot_y_2, r_elbow_rot_z], axis=2)

r_upperarm_dir = r_forearm - r_upperarm
r_upperarm_dir = r_upperarm_dir / np.linalg.norm(
    r_upperarm_dir, axis=1, keepdims=True
)

# left knee
l_calf_dir = l_calf - l_foot
l_calf_dir = l_calf_dir / np.linalg.norm(
    l_calf_dir, axis=1, keepdims=True
)
l_foot_dir = l_toe - l_foot
l_foot_dir = l_foot_dir / np.linalg.norm(
    l_foot_dir, axis=1, keepdims=True
)
l_knee_rot_y = np.cross(l_calf_dir, l_foot_dir)
l_knee_rot_y = l_knee_rot_y / np.linalg.norm(
    l_knee_rot_y, axis=1, keepdims=True
)
l_knee_rot_x = np.cross(l_knee_rot_y, l_calf_dir)
l_knee_rot_x = l_knee_rot_x / np.linalg.norm(
    l_knee_rot_x, axis=1, keepdims=True
)
l_knee_rot = np.stack([l_knee_rot_x, l_knee_rot_y, l_calf_dir], axis=2)

l_thigh_dir = l_calf - l_thigh
l_thigh_dir = l_thigh_dir / np.linalg.norm(l_thigh_dir, axis=1, keepdims=True)

# left ankle angle
cos = np.einsum('ij,ij->i', l_calf_dir, l_foot_dir) / (np.linalg.norm(l_calf_dir, axis=1) * np.linalg.norm(l_foot_dir, axis=1))
l_foot_angle = np.arccos(cos) - np.arctan(0.047/0.160) - np.pi/2

# right knee
r_calf_dir = r_calf - r_foot
r_calf_dir = r_calf_dir / np.linalg.norm(
    r_calf_dir, axis=1, keepdims=True
)
r_foot_dir = r_toe - r_foot
r_foot_dir = r_foot_dir / np.linalg.norm(
    r_foot_dir, axis=1, keepdims=True
)
r_knee_rot_y = np.cross(r_calf_dir, r_foot_dir)
r_knee_rot_y = r_knee_rot_y / np.linalg.norm(
    r_knee_rot_y, axis=1, keepdims=True
)
r_knee_rot_x = np.cross(r_knee_rot_y, r_calf_dir)
r_knee_rot_x = r_knee_rot_x / np.linalg.norm(
    r_knee_rot_x, axis=1, keepdims=True
)
r_knee_rot = np.stack([r_knee_rot_x, r_knee_rot_y, r_calf_dir], axis=2)

r_thigh_dir = r_calf - r_thigh
r_thigh_dir = r_thigh_dir / np.linalg.norm(r_thigh_dir, axis=1, keepdims=True)

# right ankle angle
cos = np.einsum('ij,ij->i', r_calf_dir, r_foot_dir) / (np.linalg.norm(r_calf_dir, axis=1) * np.linalg.norm(r_foot_dir, axis=1))
r_foot_angle = np.arccos(cos) - np.arctan(0.047/0.160) - np.pi/2


# to pybullet
upper_rot = R.from_matrix(upper_rot)
lower_rot = R.from_matrix(lower_rot)
l_knee_rot = R.from_matrix(l_knee_rot)
r_knee_rot = R.from_matrix(r_knee_rot)
l_elbow_rot = R.from_matrix(l_elbow_rot)
r_elbow_rot = R.from_matrix(r_elbow_rot)

# waist angle
R_u_l = np.einsum('ijk,ikl->ijl', np.transpose(lower_rot.as_matrix(), (0, 2, 1)), upper_rot.as_matrix())
R_u_l = R.from_matrix(R_u_l)
waist_angles = R_u_l.as_euler('xyz', degrees=False)

In [7]:
skeleton.shape

(22817, 28, 3)

In [12]:
frame_idx = 523
joints = go.Scatter3d(
    x=skeleton[frame_idx, :, 0],  
    y=skeleton[frame_idx, :, 1],  
    z=skeleton[frame_idx, :, 2], 
    mode='markers',
    marker={
        'size': 6,
        'opacity': 0.8,
    }
)

joint_idx = 22
address = go.Scatter3d(
    x=skeleton[frame_idx, joint_idx:joint_idx+1, 0],  
    y=skeleton[frame_idx, joint_idx:joint_idx+1, 1],  
    z=skeleton[frame_idx, joint_idx:joint_idx+1, 2], 
    mode='markers',
    marker={
        'size': 10,
        'opacity': 0.8,
    }
)

layout = go.Layout(
        margin={'l': 0, 'r': 0, 'b': 0, 't': 0},
        scene = dict(
            # xaxis = dict(title="x", range = [-2,0]),
            # yaxis = dict(title="y", range = [-1,1]),
            # zaxis = dict(title="z", range = [0.5,2.5]),
            aspectratio=dict(x=1, y=1, z=1)
    ))

fig = go.Figure(data=[joints, address], layout=layout)
# plotly.offline.iplot(fig)
fig.show()

In [92]:
a = np.cross(r_thigh[frame_idx] - r_calf[frame_idx], r_foot[frame_idx] - r_calf[frame_idx])
a = a / np.linalg.norm(a)
a

array([-0.2409599 ,  0.9678261 ,  0.07246283], dtype=float32)

In [30]:
l_foot[0]

array([0.10409167, 0.21831894, 0.04470253], dtype=float32)

In [93]:
r_knee_rot.as_matrix()[frame_idx, :, 1]

array([ 0.24095983, -0.96782617, -0.07246284])

In [23]:
l_knee_rot_y[0]

array([ 0.5847038 , -0.8112455 ,  0.00147046], dtype=float32)

In [146]:
origin

array([-0.1236673 ,  0.23697223,  0.41275057], dtype=float32)

In [143]:
rotation_matrix

array([[-0.0054528 ,  0.99978654, -0.01992834],
       [-0.99839719, -0.0065657 , -0.05621336],
       [-0.0563322 ,  0.01958988,  0.99821988]])

In [71]:
lower_rot.as_matrix()[frame_idx]

array([[-0.0054528 ,  0.99978654, -0.01992834],
       [-0.99839719, -0.0065657 , -0.05621336],
       [-0.0563322 ,  0.01958988,  0.99821988]])

In [24]:
urdf_path = 'adam_lite_v2/urdf/adam_lite_pybullet.urdf'
physicsClient = p.connect(p.GUI)  # non-graphical version
p.setAdditionalSearchPath(pybullet_data.getDataPath())  # used by loadURDF
robot_start_pos = [0, 0, 0]
robot_start_orientation = p.getQuaternionFromEuler([0, 0, 0])
humanoid = p.loadURDF(urdf_path, robot_start_pos, robot_start_orientation)

upperarm_len = 0.2648365892539233
lowerarm_len = 0.27628355041864106
thigh_len = 0.4252746432372357
toe_len = 0.167
heel_len = 0.079

toe_R = R.from_euler('y', np.arctan(0.047/0.160)).as_matrix()
heel_R = R.from_euler('y', np.pi / 2 + np.arctan(0.047/0.064)).as_matrix()

num_joints = p.getNumJoints(humanoid)

joint_names = []

pelvis_copy = pelvis.copy()

# Iterate through the joints and print their names
for joint_index in range(num_joints):
    joint_info = p.getJointInfo(humanoid, joint_index)
    joint_name = joint_info[1].decode(
        "utf-8"
    )  # Decode the byte string to get the joint name
    joint_names.append(joint_name)

joint_poses = []
jointDamping = [0.1] * num_joints
jointDamping[12] = 100
jointDamping[13] = 100
jointDamping[14] = 100
restPoses = [0.0] * num_joints
init_pose = [0.0] * num_joints

for i in range(num_frames):

# i = 538

    init_pose[12] = waist_angles[i,0]
    init_pose[13] = waist_angles[i,1]
    init_pose[14] = waist_angles[i,2]
    restPoses[12] = waist_angles[i,0]
    restPoses[13] = waist_angles[i,1]
    restPoses[14] = waist_angles[i,2]

    p.resetBasePositionAndOrientation(
    humanoid, (pelvis[i]).tolist(), lower_rot[i].as_quat().tolist()
    )
    for joint_index, angle in enumerate(init_pose):
        p.resetJointState(humanoid, joint_index, angle)

    l_shoulder_pos, _ = p.getLinkState(
    humanoid, joint_names.index("shoulderRoll_Left")
    )[4:6]
    r_shoulder_pos, _ = p.getLinkState(
    humanoid, joint_names.index("shoulderRoll_Right")
    )[4:6]
    l_hip_pos, _ = p.getLinkState(
    humanoid, joint_names.index("hipRoll_Left")
    )[4:6]
    r_hip_pos, _ = p.getLinkState(
    humanoid, joint_names.index("hipRoll_Right")
    )[4:6]

    # IK for l hand
    l_elbow_pos = np.array(l_shoulder_pos) + l_upperarm_dir[i] * upperarm_len

    # IK for r hand
    r_elbow_pos = np.array(r_shoulder_pos) + r_upperarm_dir[i] * upperarm_len

    # IK for l foot
    l_knee_pos = np.array(l_hip_pos) + l_thigh_dir[i] * thigh_len

    # IK for r foot
    r_knee_pos = np.array(r_hip_pos) + r_thigh_dir[i] * thigh_len

    # # Calculate the IK solution
    ik_solution_rh = p.calculateInverseKinematics(
    humanoid,
    joint_names.index("elbow_Right"),
    r_elbow_pos,
    targetOrientation=r_elbow_rot.as_quat()[i],
    jointDamping=jointDamping,  # Adjust damping as needed,
    restPoses=restPoses,  # Initial joint angles
    )

    ik_solution_lh = p.calculateInverseKinematics(
    humanoid,
    joint_names.index("elbow_Left"),
    l_elbow_pos,
    targetOrientation=l_elbow_rot.as_quat()[i],
    jointDamping=jointDamping,  # Adjust damping as needed
    restPoses=restPoses,  # Initial joint anglesss
    )

    ik_solution_rf = p.calculateInverseKinematics(
    humanoid,
    joint_names.index("kneePitch_Right"),
    r_knee_pos,
    targetOrientation=r_knee_rot.as_quat()[i],
    jointDamping=jointDamping,  # Adjust damping as needed
    restPoses=restPoses,  # Initial joint angles
    )

    ik_solution_lf = p.calculateInverseKinematics(
    humanoid,
    joint_names.index("kneePitch_Left"),
    l_knee_pos,
    targetOrientation=l_knee_rot.as_quat()[i],
    jointDamping=jointDamping,  # Adjust damping as needed
    restPoses=restPoses,  # Initial joint angles
    )

    # ik_solution = np.zeros(num_joints)
    # ik_solution[:4] = ik_solution_lf[:4]
    # ik_solution[4] = l_foot_angle[i]
    # ik_solution[6:10] = ik_solution_rf[6:10]
    # ik_solution[10] = r_foot_angle[i]
    # ik_solution[12] = waist_angles[i,0]
    # ik_solution[13] = waist_angles[i,1]
    # ik_solution[14] = waist_angles[i,2]
    # ik_solution[16:20] = ik_solution_lh[15:19]
    # ik_solution[24:28] = ik_solution_rh[19:23]

    # lite
    ik_solution = np.zeros(num_joints)
    ik_solution[:4] = ik_solution_lf[:4]
    ik_solution[4] = l_foot_angle[i]
    ik_solution[6:10] = ik_solution_rf[6:10]
    ik_solution[10] = r_foot_angle[i]
    ik_solution[12] = waist_angles[i,0]
    ik_solution[13] = waist_angles[i,1]
    ik_solution[14] = waist_angles[i,2]
    ik_solution[16:20] = ik_solution_lh[15:19]
    ik_solution[22:26] = ik_solution_rh[20:24]

    restPoses = ik_solution
    init_pose = ik_solution

    for joint_index, angle in enumerate(init_pose):
        p.resetJointState(humanoid, joint_index, angle)

    l_ankle_pos, l_ankle_rot = p.getLinkState(
    humanoid, joint_names.index("anklePitch_Left")
    )[4:6]
    r_ankle_pos, r_ankle_rot = p.getLinkState(
    humanoid, joint_names.index("anklePitch_Right")
    )[4:6]

    l_rot_matrix = p.getMatrixFromQuaternion(l_ankle_rot)
    l_rot_matrix = np.array(l_rot_matrix).reshape(3, 3)
    l_toe_pose = np.dot(l_rot_matrix@toe_R, [toe_len, 0, 0]) + l_ankle_pos
    l_heel_pose = np.dot(l_rot_matrix@heel_R, [heel_len, 0, 0]) + l_ankle_pos

    r_rot_matrix = p.getMatrixFromQuaternion(r_ankle_rot)
    r_rot_matrix = np.array(r_rot_matrix).reshape(3, 3)
    r_toe_pose = np.dot(r_rot_matrix@toe_R, [toe_len, 0, 0]) + r_ankle_pos
    r_heel_pose = np.dot(r_rot_matrix@heel_R, [heel_len, 0, 0]) + r_ankle_pos

    # sphere_radius = 0.01
    # visual_shape_id = p.createVisualShape(p.GEOM_SPHERE, radius=sphere_radius, rgbaColor=[1, 0, 0, 1])  # Red sphere
    # sphere_body_id = p.createMultiBody(baseMass=0, baseVisualShapeIndex=visual_shape_id, basePosition=l_toe_pose)
    # sphere_body_id = p.createMultiBody(baseMass=0, baseVisualShapeIndex=visual_shape_id, basePosition=l_heel_pose)
    # sphere_body_id = p.createMultiBody(baseMass=0, baseVisualShapeIndex=visual_shape_id, basePosition=r_toe_pose)
    # sphere_body_id = p.createMultiBody(baseMass=0, baseVisualShapeIndex=visual_shape_id, basePosition=r_heel_pose)

    # sphere_body_id = p.createMultiBody(baseMass=0, baseVisualShapeIndex=visual_shape_id, basePosition=r_knee_pos)
    # # sphere_body_id = p.createMultiBody(baseMass=0, baseVisualShapeIndex=visual_shape_id, basePosition=l_knee_pos)
    # # sphere_body_id = p.createMultiBody(baseMass=0, baseVisualShapeIndex=visual_shape_id, basePosition=r_toe_pose)
    # # sphere_body_id = p.createMultiBody(baseMass=0, baseVisualShapeIndex=visual_shape_id, basePosition=r_heel_pose)

    # frame_size=0.1
    # rot_matrix = r_elbow_rot.as_matrix()[i]
    # # Extract the rotated axes
    # x_axis_end = np.dot(rot_matrix, [frame_size, 0, 0]) + r_elbow_pos
    # y_axis_end = np.dot(rot_matrix, [0, frame_size, 0]) + r_elbow_pos
    # z_axis_end = np.dot(rot_matrix, [0, 0, frame_size]) + r_elbow_pos


    # p.addUserDebugLine(r_elbow_pos, x_axis_end, [1, 0, 0], parentObjectUniqueId=-1, parentLinkIndex=-1)

    # # Y-axis (green)
    # p.addUserDebugLine(r_elbow_pos, y_axis_end, [0, 1, 0], parentObjectUniqueId=-1, parentLinkIndex=-1)

    # # Z-axis (blue)
    # p.addUserDebugLine(r_elbow_pos, z_axis_end, [0, 0, 1], parentObjectUniqueId=-1, parentLinkIndex=-1)


    # frame_size=0.1
    # rot_matrix = l_elbow_rot.as_matrix()[i]
    # # Extract the rotated axes
    # x_axis_end = np.dot(rot_matrix, [frame_size, 0, 0]) + l_elbow_pos
    # y_axis_end = np.dot(rot_matrix, [0, frame_size, 0]) + l_elbow_pos
    # z_axis_end = np.dot(rot_matrix, [0, 0, frame_size]) + l_elbow_pos


    # p.addUserDebugLine(l_knee_pos, x_axis_end, [1, 0, 0], parentObjectUniqueId=-1, parentLinkIndex=-1)

    # # Y-axis (green)
    # p.addUserDebugLine(l_knee_pos, y_axis_end, [0, 1, 0], parentObjectUniqueId=-1, parentLinkIndex=-1)

    # # Z-axis (blue)
    # p.addUserDebugLine(l_knee_pos, z_axis_end, [0, 0, 1], parentObjectUniqueId=-1, parentLinkIndex=-1)

    # if i == 0:
    #     min_foot_z = np.min([l_toe_pose[2], r_toe_pose[2], l_heel_pose[2], r_heel_pose[2]])

    # pelvis_copy[i, 2] = pelvis[i,2] - min_foot_z

    # Step the simulation
    joint_poses.append(ik_solution)
    time.sleep(0.1)
    print(i)

# p.disconnect()

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 4080/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 545.29.06
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 545.29.06
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 4080/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = NVIDIA Corporation
ven = NVIDIA Corporation
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45

In [17]:
joint_names

['hipPitch_Left',
 'hipRoll_Left',
 'hipYaw_Left',
 'kneePitch_Left',
 'anklePitch_Left',
 'ankleRoll_Left',
 'hipPitch_Right',
 'hipRoll_Right',
 'hipYaw_Right',
 'kneePitch_Right',
 'anklePitch_Right',
 'ankleRoll_Right',
 'waistRoll',
 'waistPitch',
 'waistYaw',
 'shoulderBase_Left',
 'shoulderPitch_Left',
 'shoulderRoll_Left',
 'shoulderYaw_Left',
 'elbow_Left',
 'wristYaw_Left',
 'shoulderBase_Right',
 'shoulderPitch_Right',
 'shoulderRoll_Right',
 'shoulderYaw_Right',
 'elbow_Right',
 'wristYaw_Right']

In [18]:
len(joint_names)

27

In [21]:
len(ik_solution_rh)

25

In [11]:
joint_names.index("shoulderBase_Left")

15

In [23]:
joint_names[24]

'shoulderPitch_Right'

: 

In [21]:
joint_names.index("shoulderBase_Right")

23

In [86]:
r_foot_angle[i,0] * 180 / np.pi

-30.33916070825221

In [25]:
p.disconnect()

numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
destroy semaphore
semaphore destroyed
Thread TERMINATED
destroy main semaphore
main semaphore destroyed
finished
numActiveThreads = 0
btShutDownExampleBrowser stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed


In [63]:
np.arctan(0.047/0.160) * 180 / np.pi

16.37015090008355

In [87]:
np.arccos(-0.5)

2.0943951023931957

In [85]:
init_pose * 180 / np.pi

array([ 6.71163372e+00,  7.50771448e+00,  1.30669005e+02,  1.09124436e+01,
       -1.22323156e+01,  0.00000000e+00,  1.43936105e+01, -6.34846699e+00,
       -1.78459026e+02,  1.90304669e+01, -6.67029444e+00,  0.00000000e+00,
        3.25804068e-02, -4.03170932e+00, -2.15374167e+00,  0.00000000e+00,
        1.95059438e+01,  8.59846687e+01, -1.47355886e+02,  2.03940889e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
       -4.89162629e+01, -7.73326663e+01, -1.26818297e+02, -2.34664103e+01,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00])

In [142]:
v = np.load('data/A12 - crawl backwards_poses_vertices.npy')

In [140]:
np.argmin(np.linalg.norm(v[frame_idx] - np.array([-0.009,0.358,1.71]), axis=1))

5169

In [130]:
frame_idx

379